In [1]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut

out = JupyterOut.unit_square( )

http://decod.es/	v0.2.3
io loaded


# Abstraction and Discovery


The extraction of general routines from specific situations not only enables more modular, concise, and legible code, but can also reveal entirely new approaches and previously unconsidered categories of geometric form.

in computer science, abstraction is understood relatively
narrowly as a process for managing complexity by separating
a system into high-level and low-level layers

More than a management strategy, this is
an important aid for exploration and open-ended discovery in what
can otherwise seem like a rigid and deterministic creative medium.


after developing three separate fractal routines – Space-Filling
Curves, Gosper Islands, and Koch Curves – a pattern of code
emerged that suggested a common implementation. The identification
of such a commonality positions us well to extract a more generalized
routine,

While the specific mechanisms employed differed somewhat, the
Gosper Island and Koch Snowflake implementations had much in
common: both operated on collections of Segments, and controlled
subdivision through pairs of coordinates describing Points along a
normalized Segment. This subdivision was applied iteratively for each
Segment in a collection, and then repeated to generate smaller and
smaller units until some condition was satisfied

To begin developing a more general
implementation that can capture both of these earlier routines, we
can define a function that subdivides a single Segment according to
a given set of normalized division coordinates, as in the
decompose_seg() function, below.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.P08.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.P06.jpg" style="width: 200px; display: inline;">

This function appends items to a collection of sub-segments, stored in the global variable subsegs, as
directed by the global variable prms, a List of Tuples that contain
pairs of coordinates relative to a normalized Segment. Since these
two global variables need only be accessed and manipulated rather
than assigned, no global keyword is required. As before, two Vecs
are employed in the plotting of subdivision Points: uv runs along the
given Segment while vv is oriented perpendicular to it.

In [32]:
"""
Fractal Segment Decomposition
Given a Segment, returns some number of sub-segments that describe a 
fractal subdivision.
"""    
def decompose_seg(seg):
    # vecs to displace along and away from given Segment
    uv, vv = seg.vec, seg.vec.cross(UZ)
    #construct division Points by displacement and chain
    subs = Segment.chain( [seg.spt + (uv*u) + (vv*v) for u,v in prms] )
    subsegs.extend(subs)

In [33]:
"""
Main Loop for a Generalized Fractal of Segments
"""    
for n in range(gens):
    subsegs = []
    map(decompose_seg,segs)
    segs = subsegs

NameError: name 'segs' is not defined

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.P07.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.P09.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Calculation of Perpendicular Displacement Vector
Given a Segment, the two neighboring Segments, and a u,v coordinate 
describing the location of a displacement Point, this function 
calculates the Vec associated with the displacement Point. The 
resulting Point will be limited to an area bounded by the bisectors 
of the given Segment and the two adjacent Segments.
"""
def vv( seg, neis, u, v ):
    prev_seg, next_seg = neis
    fac = seg.length/2
    va = Vec.bisector(seg.vec.inverted(),prev_seg.vec).normalized(fac)
    vb = Vec.bisector(seg.vec,next_seg.vec.inverted()).normalized(fac)
    return Vec.interpolate(va,vb,u)*v

In [ ]:
"""
Bounded Fractal Segment Decomposition
Given a Segment and the two neighboring Segments, returns some number 
of sub-segments that describe a fractal subdivision that is bounded 
to the bisectors of the given Segment and the two adjacent Segments.
"""
def decompose_seg( seg, neis ):
    # vecs to displace along and away from given Segment
    uv = seg.vec
    #construct division Points by displacement and chain
    subs = Segment.chain([seg.spt+(uv*u)+vv(seg,neis,u,v) for u,v in prms])
    subsegs.extend(subs)


In [ ]:
"""
Main Loop for a Bisector-Bound Fractal of Segments 
"""
subsegs = []
for n in range(gens):
    subsegs = []
    segs_prev = [segs[-1]] + segs[:-1]
    segs_next = segs[1:] + [segs[0]]
    map( decompose_seg, segs, segs_prev, segs_next )
    segs = subsegs


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.P10.jpg" style="width: 200px; display: inline;">